In [ ]:
from cerebralcortex.util.helper_methods import get_study_names
sn = get_study_names("/home/jupyter/cc3_conf/")
print(sn)
from pyspark.sql import functions as F
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import StructField, StructType, DoubleType,MapType, StringType,ArrayType, FloatType, TimestampType, IntegerType
from pyspark.sql.functions import minute, second, mean, window
from pyspark.sql import functions as F
import numpy as np
import pandas as pd
from cerebralcortex.core.datatypes import DataStream
from cerebralcortex.core.metadata_manager.stream.metadata import Metadata, DataDescriptor, \
ModuleMetadata
from typing import List
import numpy as np
from scipy import signal
import pandas as pd
from cerebralcortex import Kernel
from pyspark.sql import functions as F
CC = Kernel("/home/jupyter/cc3_moods_conf/", study_name='moods')

In [ ]:
ppg_stream_name = 'ppg--org.md2k.watch--fossil_watch_sport'
acl_stream_name = 'accelerometer--org.md2k.watch--fossil_watch_sport'

ppg_data = CC.get_stream(ppg_stream_name)

acl_data = CC.get_stream(acl_stream_name)

ppg_data  = ppg_data.withColumn('day',F.date_format('localtime',"yyyyMMdd"))
acl_data = acl_data.withColumn('day',F.date_format('localtime',"yyyyMMdd"))

from datetime import datetime

minimum_time = datetime(2020,11,25)

acl_data = acl_data.filter(F.col('localtime')>minimum_time)
ppg_data = ppg_data.filter(F.col('localtime')>minimum_time)

In [ ]:
data = acl_data

win = F.window("timestamp", windowDuration='5 seconds',slideDuration='2 seconds',startTime='0 seconds')
groupbycols = ["user","version"] + [win]

import numpy as np
schema2 = StructType([
    StructField("version", IntegerType()),
    StructField("user", StringType()),
    StructField("start", TimestampType()),
    StructField("end", TimestampType()),
    StructField("timestamp", TimestampType()),
    StructField("localtime", TimestampType()),
    StructField("activity", DoubleType())
])
@pandas_udf(schema2, PandasUDFType.GROUPED_MAP)
def compute_std(key,data1):
    activity = np.sqrt(data1['x'].std()**2 + data1['y'].std()**2 + data1['z'].std()**2)
    temp = [data1.version.values[0],
           data1.user.values[0],
           key[2]['start'],
           key[2]['end'],
           data1['timestamp'].values[0],
           data1['localtime'].values[0],
           activity]
    return pd.DataFrame([temp],columns=['version','user','start','end','timestamp','localtime','activity'])
activity_data = data._data.groupBy(groupbycols).apply(compute_std)
schema = activity_data.schema
stream_metadata = Metadata()
stream_metadata.set_name("org.md2k.acl.std.5.secs").set_description("acl std 5 secs")
for field in schema.fields:
    stream_metadata.add_dataDescriptor(
        DataDescriptor().set_name(str(field.name)).set_type(str(field.dataType))
    )
stream_metadata.add_module(
    ModuleMetadata().set_name("acl std 5 secs") \
    .set_attribute("url", "https://md2k.org").set_author(
        "Md Azim Ullah", "mullah@memphis.edu"))
stream_metadata.is_valid()
ds = DataStream(data=activity_data,metadata=stream_metadata)
CC.save_stream(ds,overwrite=True)

In [ ]:
data = CC.get_stream("org.md2k.acl.std.5.secs")

win = F.window("timestamp", windowDuration='60 seconds',startTime='0 seconds')
groupbycols = ["user","version"] + [win]

import numpy as np
schema2 = StructType([
    StructField("version", IntegerType()),
    StructField("user", StringType()),
    StructField("start", TimestampType()),
    StructField("end", TimestampType()),
    StructField("timestamp", TimestampType()),
    StructField("localtime", TimestampType()),
    StructField("std", DoubleType())
])
@pandas_udf(schema2, PandasUDFType.GROUPED_MAP)
def compute_std(key,data1):
    activity = np.percentile(data1['activity'],25)
    temp = [data1.version.values[0],
           data1.user.values[0],
           key[2]['start'],
           key[2]['end'],
           data1['timestamp'].values[0],
           data1['localtime'].values[0],
           activity]
    return pd.DataFrame([temp],columns=['version','user','start','end','timestamp','localtime','std'])
activity_data = data._data.groupBy(groupbycols).apply(compute_std)
schema = activity_data.schema
stream_metadata = Metadata()
stream_metadata.set_name("org.md2k.acl.std.60.secs.25.percentile").set_description("acl std 60 secs 25 percentile")
for field in schema.fields:
    stream_metadata.add_dataDescriptor(
        DataDescriptor().set_name(str(field.name)).set_type(str(field.dataType))
    )
stream_metadata.add_module(
    ModuleMetadata().set_name("acl std 60 secs 25 percentile") \
    .set_attribute("url", "https://md2k.org").set_author(
        "Md Azim Ullah", "mullah@memphis.edu"))
stream_metadata.is_valid()
ds = DataStream(data=activity_data,metadata=stream_metadata)
CC.save_stream(ds,overwrite=True)

In [ ]:
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import StructField, StructType, DoubleType, StringType, TimestampType, IntegerType
import numpy as np
from cerebralcortex.core.datatypes import DataStream
from cerebralcortex.core.metadata_manager.stream.metadata import Metadata, DataDescriptor, \
    ModuleMetadata
from scipy import signal
def filter_data(X,
                Fs=100,
                low_cutoff=.4,
                high_cutoff=3.0,
                filter_order=65):
    """
    Bandpass Filter of single channel

    :param X: input data
    :param Fs: sampling freq.
    :param low_cutoff: low passband
    :param high_cutoff: high passband
    :param filter_order: no of taps in FIR filter

    :return: filtered version of input data
    """
    X1 = X.reshape(-1,1)
    X1 = signal.detrend(X1,axis=0,type='constant')
    b = signal.firls(filter_order,np.array([0,low_cutoff-.1, low_cutoff, high_cutoff ,high_cutoff+.5,Fs/2]),np.array([0, 0 ,1 ,1 ,0, 0]),
                     np.array([100*0.02,0.02,0.02]),fs=Fs)
    X2 = signal.convolve(X1.reshape(-1),b,mode='same')
    return X2

def get_metadata(data,
                 wrist='left',
                 sensor_name='motionsensehrv',
                 ppg_columns=('red','infrared','green'),
                 acl_columns=('aclx','acly','aclz')):
    """
    :param data: input stream
    :param wrist: which wrist the data was collected from
    :param sensor_name: name of sensor
    :param ppg_columns: columns in the input dataframe referring to multiple ppg channels
    :param acl_columns: columns in the input dataframe referring to accelerometer channels

    :return: metadata of output stream
    """
    stream_name = "org.md2k."+str(sensor_name)+"."+str(wrist)+".wrist.bandpass.filtered"
    stream_metadata = Metadata()
    stream_metadata.set_name(stream_name).set_description("Bandpass Filtered PPG data") \
        .add_dataDescriptor(DataDescriptor().set_name("timestamp").set_type("datetime")) \
        .add_dataDescriptor(DataDescriptor().set_name("localtime").set_type("datetime")) \
        .add_dataDescriptor(DataDescriptor().set_name("version").set_type("int")) \
        .add_dataDescriptor(DataDescriptor().set_name("user").set_type("string"))

    for c in ppg_columns:
        stream_metadata.add_dataDescriptor(DataDescriptor().set_name(c).set_type("double").set_attribute("description",
                                                                                                    "ppg channel "+c))
    for c in acl_columns:
        stream_metadata.add_dataDescriptor(DataDescriptor().set_name(c).set_type("double").set_attribute("description",
                                                                                            "accelerometer channel "+c))

    stream_metadata.add_module(
        ModuleMetadata().set_name("ecg data quality").set_attribute("url", "http://md2k.org/").set_author(
            "Md Azim Ullah", "mullah@memphis.edu"))
    return stream_metadata


def bandpass_filter(
                   data,
                   Fs = 25,
                   low_cutoff = 0.4,
                   high_cutoff = 3.0,
                   filter_order = 65,
                   ppg_columns=('red','infrared','green'),
                   acl_columns=('aclx','acly','aclz'),
                   wrist='left',
                   sensor_name='motionsensehrv'):

    """

    :param data: PPG & ACL data stream
    :param Fs: sampling frequency
    :param low_cutoff: minimum frequency of pass band
    :param high_cutoff: Maximum Frequency of pass band
    :param filter_order: no. of taps in FIR filter
    :param ppg_columns: columns in the input dataframe referring to multiple ppg channels
    :param acl_columns: columns in the input dataframe referring to accelerometer channels
    :param wrist: which wrist the data was collected from
    :param sensor_name: name of sensor

    :return: Bandpass filtered version of input PPG data
    """

    ## check if all columns exist

    default_columns = ['user','version','localtime','timestamp']
    required_columns = default_columns+acl_columns+ppg_columns
    if len(set(required_columns)-set(data.columns))>0:
        raise Exception("Columns missing in input dataframe! " + str(list(set(required_columns)-set(data.columns))))

    ## select the columns from input dataframe

    data = data.select(*required_columns)

    ## udf

    default_schema = [StructField("timestamp", TimestampType()),
                      StructField("localtime", TimestampType()),
                      StructField("version", IntegerType()),
                      StructField("user", StringType())]
    schema = StructType(default_schema+[StructField(c, DoubleType()) for c in list(ppg_columns)+list(acl_columns)])
    @pandas_udf(schema, PandasUDFType.GROUPED_MAP)
    def ppg_bandpass(data):
        data = data.sort_values('timestamp').reset_index(drop=True)
        for c in ppg_columns:
            data[c] = filter_data(data[c].values,Fs=Fs,low_cutoff=low_cutoff,high_cutoff=high_cutoff,filter_order=filter_order)
        return data

    ## steps
    ppg_bandpass_filtered = data.compute(ppg_bandpass,windowDuration=60*60*10,startTime='0 seconds')
    output_data = ppg_bandpass_filtered._data
    ds = DataStream(data=output_data,metadata=get_metadata(data,wrist=wrist,sensor_name=sensor_name,
                                                           ppg_columns=ppg_columns,acl_columns=acl_columns))
    return ds

data = CC.get_stream('ppg--org.md2k.watch--fossil_watch_sport')
data  = data.withColumn('day',F.date_format('localtime',"yyyyMMdd"))
filtered_data = bandpass_filter(
                   data,
                   Fs = 100,
                   low_cutoff = 0.4,
                   high_cutoff = 3.0,
                   filter_order = 65,
                   ppg_columns=['ppg1'],
                   acl_columns=[],
                   wrist='left',
                   sensor_name='fossil')
CC.save_stream(filtered_data,overwrite=True)

In [ ]:
from scipy.stats import iqr
data = CC.get_stream('org.md2k.fossil.left.wrist.bandpass.filtered')
# data.printSchema()
win = F.window("timestamp", windowDuration='60 seconds',startTime='0 seconds')
groupbycols = ["user","version"] + [win]

import numpy as np
schema2 = StructType([
    StructField("version", IntegerType()),
    StructField("user", StringType()),
    StructField("start", TimestampType()),
    StructField("end", TimestampType()),
    StructField("timestamp", TimestampType()),
    StructField("localtime", TimestampType()),
    StructField("iqr", DoubleType())
])
@pandas_udf(schema2, PandasUDFType.GROUPED_MAP)
def compute_std(key,data1):
    if data1.shape[0]<20*100:
        activity = 0
    else:
        activity = iqr(data1['ppg1'])
    temp = [data1.version.values[0],
           data1.user.values[0],
           key[2]['start'],
           key[2]['end'],
           data1['timestamp'].values[0],
           data1['localtime'].values[0],
           activity]
    return pd.DataFrame([temp],columns=['version','user','start','end','timestamp','localtime','iqr'])
activity_data = data._data.groupBy(groupbycols).apply(compute_std)
schema = activity_data.schema
stream_metadata = Metadata()
stream_metadata.set_name("org.md2k.ppg.filtered.iqr.60.secs").set_description("filtered ppg inter quartile range")
for field in schema.fields:
    stream_metadata.add_dataDescriptor(
        DataDescriptor().set_name(str(field.name)).set_type(str(field.dataType))
    )
stream_metadata.add_module(
    ModuleMetadata().set_name("filtered ppg inter quartile range") \
    .set_attribute("url", "https://md2k.org").set_author(
        "Md Azim Ullah", "mullah@memphis.edu"))
stream_metadata.is_valid()
ds = DataStream(data=activity_data,metadata=stream_metadata)
CC.save_stream(ds,overwrite=True)

In [ ]:
ppg_iqr_data = CC.get_stream('org.md2k.ppg.filtered.iqr.60.secs')
from datetime import datetime

minimum_time = datetime(2020,11,25)
ppg_iqr_data = ppg_iqr_data.filter(F.col('localtime')>minimum_time).drop('localtime','timestamp')

acl_std_data = CC.get_stream('org.md2k.acl.std.60.secs.25.percentile')

all_data = acl_std_data.join(ppg_iqr_data,on=['user','start','end'],how='left')
all_data = all_data.withColumn('day',F.date_format('localtime',"yyyyMMdd"))
all_data = all_data.withColumn('time',F.col('timestamp').cast('double'))

all_data = all_data.toPandas()

all_data.fillna(0,inplace=True)

all_data = all_data.sort_values('timestamp').reset_index(drop=True)
import pickle
pickle.dump(all_data,open('../data/all_data.p','wb'))

In [ ]:
import pickle
all_data = pickle.load(open('../data/all_data.p','rb'))

In [ ]:
class Sample:
    def __init__(self,acl_std=None,ppg_iqr=None,cur_time=None):
        self.last_sampled = 0
        self.acl_std = list(acl_std)
        self.ppg_iqr = [a>500 for a in ppg_iqr]
        self.cur_time = cur_time
        
    def get_decesion(self,iqr,std,cur_time):
        self.last_sampled = cur_time-self.cur_time
        self.cur_time = cur_time
        self.acl_std.append(std)
        if self.last_sampled>10*60:
            self.ppg_iqr.append(iqr>500)
            return 1
        if self.last_sampled>5*60:
            if self.ppg_iqr[-1]:
                self.ppg_iqr.append(iqr>500)
                return 1
            return -1
        temp_std = np.array(self.acl_std[-3:])
        if len(temp_std[temp_std>.0234]):
            self.ppg_iqr.append(iqr>500)
            return 1
        else:
#             self.ppg_iqr.append(False)
            return -1
            
        
        

def get_sampling_decision(df):
    df = df.sort_values('timestamp').reset_index(drop=True)
#     df['sample'] = -1
#     df.iloc[0,'sample'] = 1
#     df.iloc[1,'sample'] = 1
#     df.iloc[2,'sample'] = 1
    sample_col = [1,1,1]
    sample_class = Sample(acl_std=df['std'].loc[0:1].values,ppg_iqr=df['iqr'].loc[0:2].values,cur_time=df['time'].loc[1])
    for i,row in df.iterrows():
        if i<3:
            continue
        sample = sample_class.get_decesion(df['iqr'].loc[i],df['std'].loc[i-1],df['time'].loc[i-1])
#         df.iloc[i,'sample'] = sample
        sample_col.append(sample)
    df['sample'] = sample_col
        
    return df


temp_data1 = all_data.groupby(['user','day'],as_index=False).apply(get_sampling_decision)

In [ ]:
log_data = CC.get_stream('moods_logs')

log_data = log_data.withColumn('day',F.date_format('timestamp',"yyyyMMdd")).toPandas()

log_data['ind'] = log_data['message'].apply(lambda a:a.find('ACTIVITY INPUT'))
log_data = log_data[log_data.ind>-1]

log_data['message'] = log_data['message'].apply(lambda a:a[17:-1])

# log_data

In [ ]:
count = 0
plt.rcParams.update({'font.size':25})
import matplotlib.pyplot as plt
for user in np.unique(temp_data1['user'].values):
    for day in np.unique(temp_data1['day'].values):
        temp_data = temp_data1[(temp_data1.day==day)&(temp_data1.user==user)].sort_values('timestamp').reset_index(drop=True)
        
#         temp_log = log_data[(log_data.uuid==user)&(log_data.day==day)]
        if temp_data.shape[0]<5:
            continue
        iqr_list = np.array([0]*temp_data.shape[0])
        iqr_list[temp_data['iqr']<500] = 0
        iqr_list[temp_data['iqr']>=500] = 1
        fig,ax = plt.subplots(3,1,figsize=(20,15),sharex=True)
        ax[0].plot(temp_data['timestamp'],temp_data['sample'],'*--')
        ax[0].set_title('Sampling Decision')
    #     plt.show()
    #     plt.figure(figsize=(10,10))
        ax[1].bar(temp_data['timestamp'],temp_data['std'],.001,color='red')
        ax[1].set_title('ACL STD')
        ax[2].bar(temp_data['timestamp'],iqr_list,.001,color='green')
        ax[2].set_title('Status of PPG Data')
#         plt.setp(ax[1],xticks= temp_log['timestamp'],xticklabels=temp_log['message'])
        plt.xticks(rotation=60)
#         plt.setp(ax[2],xticks= temp_log['timestamp'],xticklabels=temp_log['timestamp'])
#         for i,row in temp_log.iterrows():
#             ax[3].text(row['timestamp'], 0.5, row['message'],fontsize=20)
        plt.savefig('../images_sampling/'+str(count)+'.pdf',dps=1e6)
        count+=1
        plt.show()
        print('-'*80)

In [ ]:
# log_data[log_data.day=='20201214']
import shutil
shutil.make_archive('../images_sampling/','zip','../images_sampling/')

In [ ]:
count

In [ ]:
schema = schema2 = StructType([
    StructField("version", IntegerType()),
    StructField("user", StringType()),
    StructField("std", DoubleType()),
    StructField("day", StringType()),
    StructField("timestamp", TimestampType()),
    StructField("localtime", TimestampType())    
])


def get_std(df_5):
    return np.sqrt(df_5['x'].std()**2+df_5['y'].std()**2+df_5['z'].std()**2)
    

def minute_level_processing(df_min):
    std_5_secs = [get_std(df_5) for i,df_5 in df_min.groupby(pd.Grouper(key='timestamp',freq='5S'))]
    return np.percentile(std_5_secs,25)

@pandas_udf(schema,PandasUDFType.GROUPED_MAP)
def temp(df):
    if df.shape[0]<100:
        return pd.DataFrame([],columns=['version','user','std','day','timestamp','localtime'])
    df_minutewise = pd.DataFrame([[1,df_min['user'].values[0],minute_level_processing(df_min),df_min['day'].values[0],df_min['timestamp'].values[0],df_min['localtime'].values[0]]
                       for i,df_min in df.groupby(pd.Grouper(key='timestamp',freq='60S')) if df_min.shape[0]>100],columns=['version','user','std','day','timestamp','localtime'])
    return df_minutewise
acl_data_sampling = acl_data._data.groupBy(['day','user','version']).apply(temp)

schema = acl_data_sampling.schema
stream_metadata = Metadata()
stream_metadata.set_name("org.md2k.acl.std.25.percentile.minute").set_description("acl std 25th percentile")
for field in schema.fields:
    stream_metadata.add_dataDescriptor(
        DataDescriptor().set_name(str(field.name)).set_type(str(field.dataType))
    )
stream_metadata.add_module(
    ModuleMetadata().set_name("ACL std 25th percentile") \
    .set_attribute("url", "https://md2k.org").set_author(
        "Md Azim Ullah", "mullah@memphis.edu"))
stream_metadata.is_valid()
ds = DataStream(data=acl_data_sampling,metadata=stream_metadata)
CC.save_stream(ds,overwrite=True)

In [ ]:
std_data = CC.get_stream("org.md2k.acl.std.25.percentile.minute")
std_data = std_data.withColumn('time',F.col('timestamp').cast('double'))
std_data = std_data.withColumn('std_final',F.array('time','std')).drop(*['std'])
win = F.window("timestamp", windowDuration='60 seconds', startTime='0 seconds')
groupbycols = ["user"] + [win]
std_data = std_data.groupby(groupbycols).agg(F.collect_list('std_final'),F.collect_list('localtime')).withColumnRenamed('collect_list(std_final)','std_final').withColumnRenamed('collect_list(localtime)','localtime')
std_data = std_data.withColumn('localtime',F.col('localtime').getItem(0))

In [ ]:
std_data.show(10,False)